In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import pandas as pd
import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata

session = cc.database.create_session()

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:34831 Dashboard: http://127.0.0.1:36573/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
input_path = '/g/data/e14/erd561/copernicus/'
output_path = '/g/data/e14/erd561/Australia_All/'

data = xr.open_dataset(input_path + 'dataset-sss-ssd-rep-monthly_1569548439236.nc')

data

<xarray.Dataset>
Dimensions:  (depth: 1, lat: 360, lon: 361, time: 300)
Coordinates:
  * depth    (depth) float32 0.0
  * lon      (lon) float32 89.875 90.125 90.375 ... 179.375 179.625 179.875
  * time     (time) object 1993-01-15 00:00:00 ... 2017-12-15 00:00:00
  * lat      (lat) float32 -69.875 -69.625 -69.375 ... 19.375 19.625 19.875
Data variables:
    sos      (time, depth, lat, lon) float32 ...
Attributes:
    Conventions:                                CF-1.7
    references:                                 Buongiorno Nardelli, B., R. D...
    institution:                                CNR
    contact:                                    servicedesk.cmems@mercator-oc...
    FROM_ORIGINAL_FILE__netcdf_version_id:      4.4.1 of Aug 10 2016 17:24:31 
    creation_date:                              Wed 17 Jul 2019 22:20:07
    FROM_ORIGINAL_FILE__product_version:        1.0
    grid_resolution:                            0.25 degrees
    FROM_ORIGINAL_FILE__software_version:       

In [5]:
# year = np.arange(2004,2018)

lonW = 88
lonE = 182

latS = -72
latN = 22

salt_z0_month = xr.open_dataset(input_path + 'dataset-sss-ssd-rep-monthly_1569548439236.nc').\
                                sos.\
                sel(lon=slice(lonW,lonE)).\
                sel(lat=slice(latS,latN)).\
                sel(depth=0, method='nearest')
print(salt_z0_month)

salt_z0 = salt_z0_month.\
                groupby('time.year').\
                mean('time')
print(salt_z0)

# salt_z0 = salt_z0.sel(year=slice(2004,2017))
# print(salt_z0)

<xarray.DataArray 'sos' (time: 300, lat: 360, lon: 361)>
[38988000 values with dtype=float32]
Coordinates:
    depth    float32 0.0
  * lon      (lon) float32 89.875 90.125 90.375 ... 179.375 179.625 179.875
  * time     (time) object 1993-01-15 00:00:00 ... 2017-12-15 00:00:00
  * lat      (lat) float32 -69.875 -69.625 -69.375 ... 19.375 19.625 19.875
Attributes:
    long_name:      sea surface salinity
    standard_name:  sea_surface_salinity
    units:          .0001
    _ChunkSizes:    [   1    1  720 1440]


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'sos' (year: 25, lat: 360, lon: 361)>
array([[[      nan,       nan,       nan, ..., 34.028866, 34.02695 ,
         34.025105],
        [      nan,       nan,       nan, ..., 34.011803, 34.01018 ,
         34.00929 ],
        [      nan,       nan,       nan, ..., 34.004814, 34.0034  ,
         34.002956],
        ...,
        [30.701935, 30.670694, 30.658201, ..., 35.02671 , 35.027878,
         35.023666],
        [30.543642, 30.53934 , 30.497835, ..., 35.048077, 35.046772,
         35.045788],
        [30.377783, 30.361895, 30.331146, ..., 35.070217, 35.066364,
         35.068134]],

       [[      nan,       nan,       nan, ..., 34.03025 , 34.029293,
         34.027363],
        [      nan,       nan,       nan, ..., 34.01235 , 34.011307,
         34.009995],
        [      nan,       nan,       nan, ..., 34.005722, 34.00488 ,
         34.003532],
        ...,
        [30.707014, 30.66664 , 30.647789, ..., 35.029514, 35.029636,
         35.02525 ],
        [30.5255

In [6]:
salt_z0_xr = xr.DataArray(np.transpose(salt_z0.values, (1,2,0)), name='salt_z0_cmems',
                        coords=[salt_z0.lat, salt_z0.lon, salt_z0.year], 
                        dims=['lat', 'lon', 'year'])
print(salt_z0_xr)
salt_z0_xr.to_netcdf(output_path + 'salt_z0_cmems.nc')

<xarray.DataArray 'salt_z0_cmems' (lat: 360, lon: 361, year: 25)>
array([[[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [34.028866, 34.03025 , 34.025146, ..., 34.031345, 34.029476,
         34.0085  ],
        [34.02695 , 34.029293, 34.024216, ..., 34.030334, 34.027325,
         34.008205],
        [34.025105, 34.027363, 34.02354 , ..., 34.02855 , 34.026196,
         34.008354]],

       [[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [34.011803, 34.01235 , 34.00774 , ..., 34.0141  , 34.01307 ,
         33.99551 ],
      

In [11]:
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

salt_z0_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='salt').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=0, method='nearest')
print(salt_z0_1_mod_month)

salt_z0_1_mod_month.load()
print(salt_z0_1_mod_month)

xt_ocean = salt_z0_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
salt_z0_1_mod_month = salt_z0_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(salt_z0_1_mod_month)

salt_z0_1_mod = salt_z0_1_mod_month.groupby('time.year').mean('time')
print(salt_z0_1_mod)

# salt_z0_1_mod = salt_z0_1_mod.sel(year=slice(2004+60*4,2017+60*4))
# print(salt_z0_1_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'salt' (time: 300, yt_ocean: 160, xt_ocean: 90)>
dask.array<getitem, shape=(300, 160, 90), dtype=float32, chunksize=(1, 124, 90), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -69.63 -69.18 -68.71 ... 17.85 18.59 19.36
  * xt_ocean  (xt_ocean) float64 -269.5 -268.5 -267.5 ... -182.5 -181.5 -180.5
    st_ocean  float64 1.152
  * time      (time) object 1958-07-02 12:00:00 ... 2257-07-02 12:00:00
Attributes:
    long_name:      Practical Salinity
    units:          psu
    valid_range:    [-10. 100.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_water_salinity
<xarray.DataArray 'salt' (time: 300, yt_ocean: 160, xt_ocean: 90)>
array([[[      nan,       nan,       nan, ..., 34.199207, 34.19863 ,
         34.197083],
        [      nan,       nan,       nan, ..., 34.169178, 34.17479 ,
         34.18189 ],
        [      nan,       nan,       nan, ..

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [12]:
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

salt_z0_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='salt').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=0, method='nearest')
print(salt_z0_025_mod_month)

salt_z0_025_mod_month.load()
print(salt_z0_025_mod_month)

xt_ocean = salt_z0_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
salt_z0_025_mod_month = salt_z0_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(salt_z0_025_mod_month)

salt_z0_025_mod = salt_z0_025_mod_month.groupby('time.year').mean('time')
print(salt_z0_025_mod)

025deg_jra55v13_iaf_gmredi6


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'salt' (time: 300, yt_ocean: 475, xt_ocean: 360)>
dask.array<getitem, shape=(300, 475, 360), dtype=float32, chunksize=(1, 216, 248), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -69.99 -69.88 -69.78 ... 19.49 19.72 19.96
  * xt_ocean  (xt_ocean) float64 -269.9 -269.6 -269.4 ... -180.6 -180.4 -180.1
    st_ocean  float64 1.152
  * time      (time) object 1958-06-30 12:00:00 ... 2257-06-30 12:00:00
Attributes:
    long_name:      Practical Salinity
    units:          psu
    valid_range:    [-10. 100.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_water_salinity
<xarray.DataArray 'salt' (time: 300, yt_ocean: 475, xt_ocean: 360)>
array([[[      nan,       nan,       nan, ..., 34.20071 , 34.19745 ,
         34.19361 ],
        [      nan,       nan,       nan, ..., 34.204533, 34.2015  ,
         34.198376],
        [      nan,       nan,       nan

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'salt' (year: 300, yt_ocean: 475, xt_ocean: 360)>
array([[[      nan,       nan,       nan, ..., 34.20071 , 34.19745 ,
         34.19361 ],
        [      nan,       nan,       nan, ..., 34.204533, 34.2015  ,
         34.198376],
        [      nan,       nan,       nan, ..., 34.20691 , 34.204037,
         34.20119 ],
        ...,
        [32.685593, 32.66067 , 32.61876 , ..., 35.244755, 35.24083 ,
         35.248047],
        [32.522778, 32.479454, 32.416817, ..., 35.286896, 35.289734,
         35.29425 ],
        [32.36751 , 32.317474, 32.289185, ..., 35.319424, 35.327473,
         35.333202]],

       [[      nan,       nan,       nan, ..., 34.188187, 34.18501 ,
         34.18151 ],
        [      nan,       nan,       nan, ..., 34.187393, 34.18515 ,
         34.183   ],
        [      nan,       nan,       nan, ..., 34.182125, 34.181103,
         34.18059 ],
        ...,
        [31.846912, 31.816566, 31.858057, ..., 35.19076 , 35.187927,
         35.189507],
    

In [13]:
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

salt_z0_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='salt').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=0, method='nearest')
print(salt_z0_01_mod_month)

salt_z0_01_mod_month.load()
print(salt_z0_01_mod_month)

xt_ocean = salt_z0_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
salt_z0_01_mod_month = salt_z0_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(salt_z0_01_mod_month)

salt_z0_01_mod = salt_z0_01_mod_month.groupby('time.year').mean('time')
print(salt_z0_01_mod)

01deg_jra55v13_iaf


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'salt' (time: 396, yt_ocean: 1186, xt_ocean: 900)>
dask.array<getitem, shape=(396, 1186, 900), dtype=float32, chunksize=(1, 300, 400), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -70.0 -69.96 -69.92 ... 19.75 19.84 19.94
  * xt_ocean  (xt_ocean) float64 -269.9 -269.8 -269.7 ... -180.2 -180.1 -180.0
    st_ocean  float64 0.5413
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
Attributes:
    long_name:      Practical Salinity
    units:          psu
    valid_range:    [-10. 100.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_water_salinity
<xarray.DataArray 'salt' (time: 396, yt_ocean: 1186, xt_ocean: 900)>
array([[[      nan,       nan,       nan, ..., 33.57218 , 33.57268 ,
         33.573517],
        [      nan,       nan,       nan, ..., 33.5636  , 33.563988,
         33.564816],
        [      nan,       nan,       

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'salt' (year: 33, yt_ocean: 1186, xt_ocean: 900)>
array([[[      nan,       nan,       nan, ..., 34.090267, 34.09027 ,
         34.089565],
        [      nan,       nan,       nan, ..., 34.089455, 34.089706,
         34.089878],
        [      nan,       nan,       nan, ..., 34.089848, 34.0899  ,
         34.089848],
        ...,
        [30.968485, 30.958977, 30.944174, ..., 35.159046, 35.16403 ,
         35.166637],
        [30.939566, 30.944902, 30.922766, ..., 35.16789 , 35.17296 ,
         35.177006],
        [30.904493, 30.900915, 30.861069, ..., 35.176414, 35.180866,
         35.185413]],

       [[      nan,       nan,       nan, ..., 34.0553  , 34.05469 ,
         34.05377 ],
        [      nan,       nan,       nan, ..., 34.05499 , 34.05455 ,
         34.05389 ],
        [      nan,       nan,       nan, ..., 34.05525 , 34.054295,
         34.05349 ],
        ...,
        [30.572237, 30.55665 , 30.585562, ..., 35.17336 , 35.174007,
         35.17507 ],
    

In [14]:
salt_z0_1_mod_xr = xr.DataArray(salt_z0_1_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='salt_z0_1_mod',
                        coords=[salt_z0_1_mod.yt_ocean, salt_z0_1_mod.xt_ocean, salt_z0_1_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
salt_z0_1_mod_xr.to_netcdf(output_path + 'salt_z0_1_mod.nc')


salt_z0_025_mod_xr = xr.DataArray(salt_z0_025_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='salt_z0_025_mod',
                        coords=[salt_z0_025_mod.yt_ocean, salt_z0_025_mod.xt_ocean, salt_z0_025_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
salt_z0_025_mod_xr.to_netcdf(output_path + 'salt_z0_025_mod.nc')


salt_z0_01_mod_xr = xr.DataArray(salt_z0_01_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='salt_z0_01_mod',
                        coords=[salt_z0_01_mod.yt_ocean, salt_z0_01_mod.xt_ocean, salt_z0_01_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
salt_z0_01_mod_xr.to_netcdf(output_path + 'salt_z0_01_mod.nc')